# Classificação de processos de homicídios
### Importando dataset gerado via webcrawler

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("MachineLearningIris") \
        .getOrCreate()

data = spark.read.format("csv").options(header='true', inferschema='true', encoding="iso-8859-1", sep=";").load('resultado_pesquisa_2021_07_10_23_49_56.csv')
data = data.na.drop()

In [4]:
drop_list = ['classe', 'comarca', 'foro', 'vara', 'data de disponibilizacao']
data = data.select([column for column in data.columns if column not in drop_list])

data.show(5)

+--------------------+--------------------+--------------------+
|             assunto|          magistrado|               texto|
+--------------------+--------------------+--------------------+
|   Homicídio Simples|Bruno Nascimento ...|TRIBUNAL DE JUSTI...|
|  Crimes de Trânsito|FABIO ANTONIO CAM...|TRIBUNAL DE JUSTI...|
|Homicídio Qualifi...|EMERSON TADEU PIR...|TRIBUNAL DE JUSTI...|
|               Furto|EDUARDO DE FRANCA...|"TRIBUNAL DE JUST...|
|  Crimes de Trânsito|Fabio Pando de Matos|TRIBUNAL DE JUSTI...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import split
from pyspark.sql.functions import size
from pyspark.sql.functions import col

dummy = data.select(split(data.texto, 'Vistos.').alias("arr"))
dummy = dummy.select(dummy.arr[size(dummy.arr)-1].alias("texto"))

dummy = dummy.select(split(dummy.texto, 'Intime-se').alias("arr"))
dummy = dummy.select(dummy.arr[size(dummy.arr)-1].alias("d_text"))

data = data.filter(~col("assunto").contains("Homicídio"))
data.show(5)

+------------------+--------------------+--------------------+
|           assunto|          magistrado|               texto|
+------------------+--------------------+--------------------+
|Crimes de Trânsito|FABIO ANTONIO CAM...|TRIBUNAL DE JUSTI...|
|             Furto|EDUARDO DE FRANCA...|"TRIBUNAL DE JUST...|
|Crimes de Trânsito|Fabio Pando de Matos|TRIBUNAL DE JUSTI...|
|             Roubo|MARIA PRISCILLA E...|"TRIBUNAL DE JUST...|
|     Crime Culposo|LEONARDO PRAZERES...|TRIBUNAL DE JUSTI...|
+------------------+--------------------+--------------------+
only showing top 5 rows



In [10]:
from pyspark.sql.functions import col
data.groupBy("assunto") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
|             assunto|count|
+--------------------+-----+
|  Crimes de Trânsito| 5644|
|Tráfico de Drogas...| 5556|
|               Roubo| 3479|
|Indenização por D...| 1834|
|Crimes do Sistema...| 1515|
|   Furto Qualificado| 1141|
|Crimes contra a vida|  970|
|               Furto|  951|
|Indenização por D...|  627|
|Acidente de Trânsito|  612|
|          Receptação|  560|
|      Lesão Corporal|  533|
|          Latrocínio|  508|
|Decorrente de Vio...|  469|
|      Roubo Majorado|  467|
|              Ameaça|  374|
|                Leve|  307|
|Uso de documento ...|  241|
|         Estelionato|  233|
|Responsabilidade ...|  212|
+--------------------+-----+
only showing top 20 rows



In [11]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="texto", outputCol="words", pattern="\\W")
# stop words
add_stopwords = ["de","a","o","que","e","do","da","em","um","para","é","com","não","uma","os","no","se","na",
                 "por","mais","as","dos","como","mas","foi","ao","ele","das","tem","à","seu","sua","ou","ser",
                 "quando","muito","há","nos","já","está","eu","também","só","pelo","pela","até","isso","ela",
                 "entre","era","depois","sem","mesmo","aos","ter","seus","quem","nas","me","esse","eles","estão",
                 "você","tinha","foram","essa","num","nem","suas","meu","às","minha","têm","numa","pelos","elas",
                 "havia","seja","qual","será","nós","tenho","lhe","deles","essas","esses","pelas","este","fosse",
                 "dele","tu","te","vocês","vos","lhes","meus","minhas","teu","tua","teus","tuas","nosso","nossa",
                 "nossos","nossas","dela","delas","esta","estes","estas","aquele","aquela","aqueles","aquelas",
                 "isto","aquilo","estou","está","estamos","estão","estive","esteve","estivemos","estiveram",
                 "estava","estávamos","estavam","estivera","estivéramos","esteja","estejamos","estejam",
                 "estivesse","estivéssemos","estivessem","estiver","estivermos","estiverem","hei","há","havemos",
                 "hão","houve","houvemos","houveram","houvera","houvéramos","haja","hajamos","hajam","houvesse",
                 "houvéssemos","houvessem","houver","houvermos","houverem","houverei","houverá","houveremos",
                 "houverão","houveria","houveríamos","houveriam","sou","somos","são","era","éramos","eram","fui",
                 "foi","fomos","foram","fora","fôramos","seja","sejamos","sejam","fosse","fôssemos","fossem",
                 "for","formos","forem","serei","será","seremos","serão","seria","seríamos","seriam","tenho",
                 "tem","temos","tém","tinha","tínhamos","tinham","tive","teve","tivemos","tiveram","tivera",
                 "tivéramos","tenha","tenhamos","tenham","tivesse","tivéssemos","tivessem","tiver","tivermos",
                 "tiverem","terei","terá","teremos","terão","teria","teríamos","teriam"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "assunto", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)
dataset.show(5)

+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|           assunto|          magistrado|               texto|               words|            filtered|            features|label|
+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|Crimes de Trânsito|FABIO ANTONIO CAM...|TRIBUNAL DE JUSTI...|[tribunal, de, ju...|[tribunal, justi,...|(10000,[0,1,2,3,4...|  0.0|
|             Furto|EDUARDO DE FRANCA...|"TRIBUNAL DE JUST...|[tribunal, de, ju...|[tribunal, justi,...|(10000,[0,1,2,3,4...|  7.0|
|Crimes de Trânsito|Fabio Pando de Matos|TRIBUNAL DE JUSTI...|[tribunal, de, ju...|[tribunal, justi,...|(10000,[0,1,2,3,4...|  0.0|
|             Roubo|MARIA PRISCILLA E...|"TRIBUNAL DE JUST...|[tribunal, de, ju...|[tribunal, justi,...|(10000,[0,1,2,3,4...|  2.0|
|     Crime Culposo|LEONARDO PRAZERES...|TRIBUNAL DE JUSTI...|[tribunal, de,

In [13]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)

In [ ]:
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("texto","assunto","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)